Напишем код, который автоматически, на основании xls файлов, содержащих банковские выписки, а так же csv файла 'additional_information',  рассчитывает комиссию, которую забтрает себе эйкваринг-компания, и формирует единый отчет об операциях, содержащий всю необходимую информацию о платежах, собранную из всех доступных файлов. Отчет затем сохраняется в виде xlsx файла.

In [13]:
import numpy as np
import pandas as pd
import os
from datetime import datetime

# исходная папка с данными:
folder = "C:/Users/Ruslan/Desktop/Расчет комиссии/"
list_of_statements = [] # список, в который считаем датафреймы с разными
# выписками из директории 'Выписки'

# В цикле по всем xls файлам из директории считываем каждый из них 
# в качестве Pandas DataFrame и добавляем в конец списка датафреймов
for dirname, _, files in os.walk(folder+'Выписки/'):
    for file in files:
        path = os.path.join(dirname,file)
        list_of_statements.append(pd.read_excel(path))

        
statements_df = pd.concat(list_of_statements, axis = 0) #объединение всех датафреймов в один, 
# содержащий информацию из всех выписок
statements_df.reset_index(drop=True, inplace=True) # индексация датафрейма заново

statements_df

,Номер устройства,Дата операции,Дата обработки,Сумма операции,Торговая уступка,К перечислению,РРН,Тип операции,Код авторизации,Номер платежного поручения,Дата платежного поручения
0,122,21.03.2023 06:05:19,21.03.2023 06:37:51,22117,249.92,21867.08,63467,Покупка,JN78,1,22.03.2023
1,122,21.03.2023 07:41:06,21.03.2023 08:08:48,9320,105.32,9214.68,80014,Покупка,XH85,1,22.03.2023
2,122,20.03.2023 23:14:14,21.03.2023 09:13:58,105707,1194.49,104512.51,47214,Покупка,RC99,1,22.03.2023
3,122,21.03.2023 08:56:33,21.03.2023 09:18:09,5441,61.48,5379.52,747,Покупка,IY191,1,22.03.2023
4,122,20.03.2023 20:45:31,21.03.2023 09:38:19,55327,625.20,54701.80,16261,Покупка,JD135,1,22.03.2023
...,...,...,...,...,...,...,...,...,...,...,...
107,122,22.03.2023 01:22:48,22.03.2023 04:46:25,28096,317.48,27778.52,27612,Покупка,OE29,1,22.03.2023
108,122,21.03.2023 22:38:30,22.03.2023 04:50:15,100935,1140.57,99794.43,13836,Покупка,BL18,1,22.03.2023
109,122,22.03.2023 03:59:28,22.03.2023 06:05:28,309982,3502.80,306479.20,31191,Покупка,EO111,1,22.03.2023
110,123,20.03.2023 19:05:55,21.03.2023 09:43:17,76107,860.01,75246.99,85432,Покупка,LN15,2,22.03.2023


In [14]:
# считываем данные из uni.xls в датафрейм uni_df:
additional_information_df = pd.read_csv(folder+'additional_information.csv', 
                                        sep = ';', encoding = '1251') 
additional_information_df



,OrderID,DateTime,MinDateTime,MaxDateTime,Status,AuthCode,Currency
0,1846-3,22.11.2022 13:32,22.11.2022 13:32,01.03.2023 16:46,Оплачен,JN78,RUB
1,5616-4,07.12.2022 16:14,07.12.2022 16:14,01.03.2023 16:16,Возврат средств,XH85,RUB
2,7755-15,10.12.2022 22:54,10.12.2022 22:54,01.03.2023 16:16,Возврат средств,RC99,RUB
3,3674-3,15.12.2022 21:43,15.12.2022 21:45,01.03.2023 17:52,Оплачен,IY191,RUB
4,7217-18,01.01.2023 23:28,01.01.2023 23:28,01.03.2023 10:06,Оплачен,JD135,RUB
...,...,...,...,...,...,...,...
2990,5898-74,22.03.2023 1:07,22.03.2023 1:12,22.03.2023 1:34,Оплачен,RC782,RUB
2991,3111-36,22.03.2023 1:22,22.03.2023 1:22,22.03.2023 9:34,Оплачен,UE456,RUB
2992,2141-35,22.03.2023 1:22,22.03.2023 1:22,22.03.2023 2:04,Оплачен,BY786,RUB
2993,3443-70,22.03.2023 3:58,22.03.2023 3:58,22.03.2023 6:04,Оплачен,SH721,RUB


Далее удалим символ '№', который может оказаться на первом месте в поле 'OrderID', а так же перенесем инрформацию об orderID из датафрейма 'additional_information_df' в исходный датафрейм. Рассчитаем так же комиссию и итоговую сумму перевода за вычетом комиссии, которую эйкваринг компания переведет клиенту.

In [15]:
# Удаление символа '№' из поля 'Код авторизации', если этот символ окажется на первом месте:
statements_df['Код авторизации'] = statements_df['Код авторизации'].apply(lambda x: x[1:] if x[0]=='№' else x)

# Добавление 'orderID', найденного  в дата фрейме uni по полю AuthCode: 
statements_df=pd.merge(statements_df,additional_information_df[['AuthCode','OrderID']], left_on='Код авторизации',
                       right_on='AuthCode', how = 'left')
statements_df.drop(columns='AuthCode', inplace=True)

# Добавление текущего времени в качестве нового столбца,
# текущее время замеряется единожды перед операцией добавления столбца и общее для всех записей:
time = datetime.now().strftime("%d.%m.%Y %H:%M")
statements_df['Время обработки'] = time

# Рассчет столбцов 'Комиссия', 'Сумма к переводу':
statements_df['Комиссия'] = statements_df['Сумма операции']*0.0125
statements_df['Сумма к переводу'] = statements_df['Сумма операции'] - statements_df['Комиссия']

# Округление необходимых столбцов до двух знаков после запятой:
columns_to_round = ['Комиссия','Сумма к переводу','Сумма операции']
statements_df[['Комиссия','Сумма к переводу','Сумма операции']] = statements_df[['Комиссия','Сумма к переводу','Сумма операции']].apply(lambda x: round(x,2))

# Создание отчета и его запись в исходную директорию в виде xlsx файла:
columns_in_report = ['Номер устройства', 
                     'Дата операции', 
                     'Дата обработки',
                     'Сумма операции',
                     'Торговая уступка',
                     'К перечислению',
                     'РРН', 
                     'Тип операции', 
                     'Код авторизации',
                     'Время обработки',
                     'OrderID',
                     'Комиссия',
                     'Сумма к переводу',
                     'Номер платежного поручения',
                     'Дата платежного поручения']
report_df = statements_df[columns_in_report]
report_df.rename(columns={'OrderID': 'orderID'}, inplace = True)
report_df.to_excel(folder+'\Отчет.xlsx')

Итоговый отчет:

In [16]:
report_df

,Номер устройства,Дата операции,Дата обработки,Сумма операции,Торговая уступка,К перечислению,РРН,Тип операции,Код авторизации,Время обработки,orderID,Комиссия,Сумма к переводу,Номер платежного поручения,Дата платежного поручения
0,122,21.03.2023 06:05:19,21.03.2023 06:37:51,22117,249.92,21867.08,63467,Покупка,JN78,05.12.2023 17:58,1846-3,276.46,21840.54,1,22.03.2023
1,122,21.03.2023 07:41:06,21.03.2023 08:08:48,9320,105.32,9214.68,80014,Покупка,XH85,05.12.2023 17:58,5616-4,116.50,9203.50,1,22.03.2023
2,122,20.03.2023 23:14:14,21.03.2023 09:13:58,105707,1194.49,104512.51,47214,Покупка,RC99,05.12.2023 17:58,7755-15,1321.34,104385.66,1,22.03.2023
3,122,21.03.2023 08:56:33,21.03.2023 09:18:09,5441,61.48,5379.52,747,Покупка,IY191,05.12.2023 17:58,3674-3,68.01,5372.99,1,22.03.2023
4,122,20.03.2023 20:45:31,21.03.2023 09:38:19,55327,625.20,54701.80,16261,Покупка,JD135,05.12.2023 17:58,7217-18,691.59,54635.41,1,22.03.2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,122,22.03.2023 01:22:48,22.03.2023 04:46:25,28096,317.48,27778.52,27612,Покупка,OE29,05.12.2023 17:58,3082-1,351.20,27744.80,1,22.03.2023
108,122,21.03.2023 22:38:30,22.03.2023 04:50:15,100935,1140.57,99794.43,13836,Покупка,BL18,05.12.2023 17:58,8432-18,1261.69,99673.31,1,22.03.2023
109,122,22.03.2023 03:59:28,22.03.2023 06:05:28,309982,3502.80,306479.20,31191,Покупка,EO111,05.12.2023 17:58,1371-6,3874.78,306107.22,1,22.03.2023
110,123,20.03.2023 19:05:55,21.03.2023 09:43:17,76107,860.01,75246.99,85432,Покупка,LN15,05.12.2023 17:58,8404-8,951.34,75155.66,2,22.03.2023
